In [1]:
from neo4j import GraphDatabase

# === Configuration ===
NEO4J_URI = "neo4j+ssc://7bc3d0ce.databases.neo4j.io"  # or your cloud URI
NEO4J_USER = "neo4j"
NEO4J_PASSWORD = "5Yn5t-lOXiZ_PfsM1AN09HYoL9aTrwC-rdSHaQP8FHw"

In [2]:
from neo4j import GraphDatabase
import pandas as pd
import csv

class Neo4jConnection:
    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))
    
    def close(self):
        self.driver.close()
    
    def execute_query(self, query, parameters=None):
        with self.driver.session() as session:
            result = session.run(query, parameters)
            return result.data()

In [3]:
def create_nodes_from_csv(conn, csv_file, node_label, node_id_column):
    """Create nodes from CSV file"""
    df = pd.read_csv(csv_file)
    
    for _, row in df.iterrows():
        # Create node with all properties from CSV row
        properties = {k: v for k, v in row.items() if pd.notna(v)}
        
        query = f"""
        MERGE (n:{node_label} {{{node_id_column}: $id}})
        SET n += $properties
        """
        
        conn.execute_query(query, {
            'id': row[node_id_column],
            'properties': properties
        })

def create_relationships_from_csv(conn, csv_file, from_label, to_label, 
                                 from_id_col, to_id_col, rel_type):
    """Create relationships from CSV file"""
    df = pd.read_csv(csv_file)
    try:
        for _, row in df.iterrows():
            query = f"""
            MATCH (a:{from_label} {{{from_id_col}: $from_id}})
            MATCH (b:{to_label} {{{to_id_col}: $to_id}})
            CREATE (a)-[r:{rel_type}]->(b)
            """
            
            conn.execute_query(query, {
                'from_id': row[from_id_col],
                'to_id': row[to_id_col]
            })
    except Exception as e:
        print(e)

# Usage example
conn = Neo4jConnection(NEO4J_URI, "neo4j", NEO4J_PASSWORD)



In [14]:
# Create nodes from CSV
create_nodes_from_csv(conn, "knowledge_graph_tables/Product.csv", "Product", "Product_ID")
create_nodes_from_csv(conn, "knowledge_graph_tables/Location_Product.csv", "Location_Product", "Loc_Product_ID")


In [15]:
create_nodes_from_csv(conn, "knowledge_graph_tables/Location_Resource.csv", "Location_Resource", "Loc_Resource_ID")
create_nodes_from_csv(conn, "knowledge_graph_tables/Location_Source.csv", "Location_Source", "Location_ID")
create_nodes_from_csv(conn, "knowledge_graph_tables/Production_Source_Header.csv", "Production_Source_Header", "Source_ID")
create_nodes_from_csv(conn, "knowledge_graph_tables/Resource.csv", "Resource", "Resource_ID")
create_nodes_from_csv(conn, "knowledge_graph_tables/UoM_Conversion_Factor.csv", "UoM_Conversion_Factor", "Product_ID")

In [16]:
create_relationships_from_csv(conn, "knowledge_graph_tables/Location_Source.csv", "Location_Source", "Product", 
                            "Location_ID",  "Product_ID", "LOCATION_SOURCE_SUPPLIES_TO_PRODUCT")

In [17]:
create_relationships_from_csv(conn, "knowledge_graph_tables/Location_Source.csv", "Location_Source", "Location_Product", 
                            "Ship_From_Loc_ID",  "Product_ID", "LOCATION_SOURCE_APPLIES_TO_LOCATION_PRODUCT")

In [18]:
create_relationships_from_csv(conn, "knowledge_graph_tables/UoM_Conversion_Factor.csv", "UoM_Conversion_Factor", "Product", 
                             "Target_UoM", "Product_ID", "UOM_CONVERSION_FACTOR_RELATES_TO_PRODUCT")

In [19]:
create_relationships_from_csv(conn, "knowledge_graph_tables/Production_Source_Header.csv", "Production_Source_Header", "Product", 
                             "Source_ID", "Product_ID", "PRODUCTION_SOURCE_HEADER_PRODUCES_FOR_PRODUCT")

In [20]:
create_relationships_from_csv(conn, "knowledge_graph_tables/Location_Product.csv", "Location_Product", "Product", 
                            "Loc_Product_ID", "Product_ID",  "LOCATION_DETAILS_FOR_PRODUCT")

In [21]:
create_relationships_from_csv(conn, "knowledge_graph_tables/Production_Source_Header.csv","Production_Source_Header", "Location_Source",  
                            "Source_ID", "Location_ID",  "PRODUCTION_SOURCE_HEADER_PRODUCES_LOCATION_PRODUCT")

In [22]:
create_relationships_from_csv(conn, "knowledge_graph_tables/Location_Resource.csv","Location_Resource", "Location_Source",  
                            "Loc_Resource_ID", "Location_ID",  "LOCATION_RESOURCE_APPLIES_TO_LOCATION_SOURCE")

In [24]:
create_relationships_from_csv(conn, "knowledge_graph_tables/Location_Resource.csv","Location_Resource", "Resource",  
                            "Location_ID", "Resource_ID", "LOCATION_RESOURCE_USES_RESOURCE")

In [29]:
create_nodes_from_csv(conn, "knowledge_graph_tables/Demand_Fulfillment.csv", "Demand_Fulfillment", "delivery_record_id")
create_nodes_from_csv(conn, "knowledge_graph_tables/Review_Capacity.csv", "Review_Capacity", "location_resource_id")
create_nodes_from_csv(conn, "knowledge_graph_tables/Review_Component.csv", "Review_Component", "product_location_id")
create_nodes_from_csv(conn, "knowledge_graph_tables/Review_DC.csv", "Review_DC", "delivery_record_id")
create_nodes_from_csv(conn, "knowledge_graph_tables/Review_Plant.csv", "Review_Plant", "product_location_id")
create_nodes_from_csv(conn, "knowledge_graph_tables/Review_Vendors.csv", "Review_Vendors", "product_location_id")
create_nodes_from_csv(conn, "knowledge_graph_tables/Profit_Margin_ts.csv", "Profit_Margin_ts", "product_date_id")

In [39]:
create_relationships_from_csv(conn, "knowledge_graph_tables/Review_Vendors.csv","Review_Vendors", "Review_Plant",  
                            "product_location_id", "Location_ID", "VENDOR_SUPPLIES_RAW_MATERIALS")

In [31]:
create_relationships_from_csv(conn, "knowledge_graph_tables/Review_Component.csv","Review_Component", "Review_Plant",  
                            "product_location_id", "Location_ID", "RAW_MATERIAL_SUPPLY_TO_PLANT")

In [32]:
create_relationships_from_csv(conn, "knowledge_graph_tables/Review_Capacity.csv","Review_Capacity", "Review_Plant",  
                            "location_resource_id", "Location_ID", "RESOURCE_CAPACITY_FOR_PLANT")

In [33]:
create_relationships_from_csv(conn, "knowledge_graph_tables/Demand_Fulfillment.csv","Review_DC", "Demand_Fulfillment",  
                            "Customer_Receipts", "Consensus_Demand", "DC_SUPPLIES_PRODUCT_TO_CUSTOMER")

In [4]:
create_relationships_from_csv(conn, "knowledge_graph_tables/Review_Plant.csv","Review_Plant", "Review_DC",  
                            "Location_ID", "Product_ID", "PLANT_SUPPLIES_PRODUCT_TO_DC")

{code: Neo.ClientError.Transaction.TransactionHookFailed} {message: You have exceeded the logical size limit of 400000 relationships in your database (attempt to add 16875 relationships would reach 410834 relationships). Please consider upgrading to the next tier.}
